In [5]:
import googlemaps

In [6]:
API_KEY = open("./API_KEY.txt").read()
map_client = googlemaps.Client(API_KEY)

In [7]:
my_location = map_client.geolocate()

In [8]:
colleg_loc = (30.0263349,31.2094813) # الكلية
helwan_metro_loc = (29.8489866,31.3320422) # مترو حلوان

In [ ]:
'''
Useful params in places_nearby
    max_price [0 , 4] "4 is most expensive"
    type
    radius
'''
res = map_client.places_nearby(location=(29.8489139,31.3360409), radius=1000, type="clothing_store")

In [10]:
'''
distance API:
    destination_addresses
    distance in km
    duration
'''
# map_client.distance_matrix((29.8131478,31.3031655), helwan_metro_loc)

'\ndistance API:\n    destination_addresses\n    distance in km\n    duration\n'

In [11]:
'''
name
opening_hours : {'open_now': True}
vicinity
rating
user_ratings_total
geometry
price_level
'''

"\nname\nopening_hours : {'open_now': True}\nvicinity\nrating\nuser_ratings_total\ngeometry\nprice_level\n"

In [12]:
# for x in res["results"]:
#     if "restaurant" in x["types"]:
#         print("Restaurant: ", x["name"])
#     elif "hospital" in x["types"]:
#         print("Hospital: ", x["name"])
#     elif "lodging" in x["types"]:
#         print("Lodging: ", x["name"])
#     elif "university" in x["types"]:
#         print("University: ", x["name"])
#     elif "travel_agency" in x["types"]:
#         print("Travel agency: ", x["name"])
#     elif "doctor" in x["types"]:
#         print("Doctor: ", x["name"])
#     elif "clothing_store" in x["types"]:
#         print("Clothes: ", x["name"])


In [19]:
class location_recommender:
    def __init__(self) -> None:
        self._API_KEY = open("./API_KEY.txt").read()
        self.map_client = googlemaps.Client(self._API_KEY)
        pass

    def recommend(self, my_location, categ, radius=1000, is_rankby_distange= False):
        # @categ => [restaurant, clothing_store, hospital, doctor, ...]

        results = []
        query = None

        try:
            if is_rankby_distange == False:
                query = self.map_client.places_nearby(my_location, radius=radius, type=categ)
            else:
                query = self.map_client.places_nearby(my_location, type=categ, rank_by="distance")

            query = query["results"]

            for res in query:
                new_location = (res["geometry"]["location"]["lat"], res["geometry"]["location"]["lng"])
                new_entry = {
                    "geometry": res["geometry"],
                    "name": res["name"],
                    "distance": self.map_client.distance_matrix(my_location, new_location),
                    "business_status": res["business_status"],
                    "types": res["types"],
                    "vicinity": res["vicinity"]
                }

                if "price_level" in res.keys(): new_entry["price_level"] = res["price_level"]
                if "rating" in res.keys(): new_entry["rating"] = res["rating"]
                if "user_ratings_total" in res.keys(): new_entry["user_ratings_total"] = res["user_ratings_total"]

                results.append(new_entry)

            return results

        except:
            return []


In [20]:
model = location_recommender()
# res = model.recommend(colleg_loc, "restaurant", 1000)

In [21]:
import dill as pickle

path = "../utils/location_recommender"

with open(path ,'wb') as f:
    pickle.dump(model, f)